In [2]:
import os
import sys
import yaml
from tqdm import tqdm
import re
import itertools as it

In [3]:
sys.path.append(os.path.dirname(os.getcwd()))

In [4]:
from config import INTENTS, ENTITIES, UTTERANCES

In [5]:
ROOT_DIR = '/home/h4zzkr/lab/delOS/'

In [35]:
def get_stopwords():
    path = os.path.join(ROOT_DIR, 'data/resources/stopwords.txt')
    words = open(path, 'r').read().split('\n')[:-1]
    return words

In [31]:
import os
# os.environ["MODEL_DIR"] = '../model'
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action
import nlpaug.flow as naf



In [8]:
import os
import yaml
import tqdm
import argparse
import re
import pandas as pd
from pathlib import Path
from config import ROOT_DIR, INTENTS, ENTITIES, UTTERANCES
#TODO: убрать комбинации шаблонных, аугментации, мб на основе рандома решать, какие слоты заменять синонимами

class DatasetBuilder:
    def __init__(self, path2dset, max_synonyms, path2write=None, drop_stopwords=False):
        # TODO: add synonym augmentions
        # TODO: classes normalization: too many classes of ones
        self.path = Path(os.path.join(ROOT_DIR, path2dset))
        self.out = os.path.join(ROOT_DIR, path2write)
        self.max_synonyms = max_synonyms
        self.intent_vocab = []
        self.tag_vocab = []

    def write_intent_vocab(self):
        with open(os.path.join(self.path.parent, 'vocab.intent'), 'w') as file:
            for i in self.intent_vocab:
                file.write(i + '\n')

    def write_tag_vocab(self):
        with open(os.path.join(self.path.parent, 'vocab.tag'), 'w') as file:
            for i in self.tag_vocab:
                file.write('B-' + i + '\n')
                file.write('I-' + i + '\n')

    def augment_intent(slot_map, intent):
        pass

        
    def build_dataset(self):
        dset = self.read_yaml()
        raw_intents, raw_entities = dset[INTENTS], dset[ENTITIES]
        self.entities_vocab = self.build_entities_vocab(raw_entities)
        print('building maps for intent tempaltes...')
        processed_intents = self.read_slots_from_yaml(raw_intents)
        print('starting to combine entities...')
        df = self.build_combination_sequence(processed_intents)
        print(f'{len(df)} unique examples builded')
        self.write_intent_vocab(); self.write_tag_vocab()
        if self.out:
            df.to_csv(self.out, index=False)
        else:
            return df
        
    def read_yaml(self):
        with open(self.path) as file:
            data = yaml.load(file, Loader=yaml.FullLoader)
        return data
        
    def build_entities_vocab(self, raw_entities):
        vocab = {}
        for e in raw_entities:
            values = []
            for v in e['values']:
                values.append(v[:self.max_synonyms])
            name = f"{e['type']}:{e['name']}"
            vocab.update({name : values})
            self.tag_vocab.append(name)
        return vocab
    
    def read_slots_from_yaml(self, raw_intents):
        intents = []
        for intent_class in raw_intents:
            intent_name = intent_class['name']
            self.intent_vocab.append(intent_name)
            for i in intent_class[UTTERANCES]:
                tag_class_ids = [j.span() for j in re.finditer(r'\[(.*?)\]', i)]
                tag_class, tag_names = [], []
                for t in tag_class_ids:
                    tag_class.append(i[t[0]:t[1]].replace('[', '').replace(']', ''))
                i = re.sub(r'\[(.*?)\]', '', i)
                tag_ids = [j.span() for j in re.finditer(r'\((.*?)\)', i)]
                for t in tag_ids:
                    tag_names.append(i[t[0]:t[1]].replace('(', '').replace(')', ''))
                tag_map = list(zip(tag_class, tag_names))
                intent_pair = [intent_name, i, tag_map]
                intents.append(intent_pair)
        return intents
    
    def augment(self, text):
        aug = naw.ContextualWordEmbsAug(
        model_path='bert-base-uncased', action="insert")
        augmented_text = aug.augment(text)
        print("Original:")
        print(text)
        print("Augmented Text:")
        print(augmented_text)

    def make_line(self, iclass, intent, imap):
        intent_len = len(intent.split())
        mask = ['O'] * intent_len
        for key in imap:
            slot_class, slot = key
            # pos =
            pos = [j.span() for j in re.finditer(f'({slot})', intent)][0]
            pos = len(intent[:pos[0]].split()) - 1
            slot_len = len(slot.split())
            if slot_len > 1:
                mask[pos] = 'B-' + slot_class
                for i in range(1, slot_len):
                    mask[pos+1] = 'I-' + slot_class
            else:
                mask[pos] = 'B-' + slot_class
        text = intent.replace('(', '').replace(')', '')
        mask = ' '.join(mask)
        return (iclass, text, mask, intent_len)
                              
            
    def build_combination_sequence(self, p_intents):
        df = pd.DataFrame(columns=['intent_label', 'words', 'word_labels', 'length'])
        cnter = 0
        for i in p_intents:
            intent_class, intent, map_ = i
            df.loc[cnter] = self.make_line(intent_class, intent, map_)
            cnter += 1

        return df

In [47]:
text = 'enable lights in the bedroom'

In [36]:
aug = naf.Sequential([
    naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="insert"),
    naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="substitute")
])

In [48]:
augs = []
cnt = 0
aug_num = 10
while cnt != aug_num:
    aug_text = aug.augment(text)
#     if 'i ' in aug_text or ' they ' in aug_text or ' we ' in aug_text:
#         pass
#     else:
    augs.append(aug_text)
    cnt += 1
print(augs)

['enable outdoor lights in the park', 'enable lights blazing into the bedroom', 'enable lights in the spare bedroom', 'enable additional lights in his bedroom', 'enable lights in the second .', 'the lights on in the bedroom', 'enable clothing allowed in the bedroom', 'the lights in all the bedroom', 'enable lights on in his bedroom', 'enable it on in the bedroom']


In [9]:
builder = DatasetBuilder(path2dset='data/nlu_data/custom/dataset.yaml',
                        max_synonyms=3,
                        path2write='data/nlu_data/custom/train.csv')
builder.build_dataset()

building maps for intent tempaltes...
starting to combine entities...
33 unique examples builded


In [57]:
custom = pd.read_csv('/home/h4zzkr/lab/delOS/data/nlu_data/custom/train.csv')
snips = pd.read_csv('/home/h4zzkr/lab/delOS/data/nlu_data/merged/train.csv')
custom = custom.reset_index()
df = pd.concat((snips, custom))
# df.to_csv('/home/h4zzkr/lab/delOS/data/nlu_data/custom2/train.csv')

In [72]:
# df = df.drop(columns=['Unnamed: 0'])
df = df.drop(columns=['index'])

In [77]:
df = df.reset_index()

ValueError: cannot insert level_0, already exists

In [76]:
df.length = df.length.astype(int)

In [80]:
df.columns

Index(['index', 'level_0', 'intent_label', 'words', 'word_labels', 'length'], dtype='object')

In [81]:
df = df.drop(columns=['index', 'level_0'])

In [82]:
df = df.reset_index()

In [83]:
df

,index,intent_label,words,word_labels,length
0,0,AddToPlaylist,Add Don and Sherri to my Meditate to Sounds of...,O B-entity_name I-entity_name I-entity_name O ...,12
1,1,AddToPlaylist,put United Abominations onto my rare groove pl...,O B-entity_name I-entity_name O B-playlist_own...,8
2,2,AddToPlaylist,add the tune by misato watanabe to the Trapeo ...,O O B-music_item O B-artist I-artist O O B-pla...,10
3,3,AddToPlaylist,add this artist to my this is miguel bosé play...,O O B-music_item O B-playlist_owner B-playlist...,10
4,4,AddToPlaylist,add heresy and the hotel choir to the evening ...,O B-entity_name I-entity_name I-entity_name I-...,11
...,...,...,...,...,...
13161,13161,turnLightOn,turn back on lights in the loft,O O O O O O B-location:room,7
13162,13162,turnLightOn,turn on lights for the little kitchen,O O O O O O B-location:room,7
13163,13163,turnLightOn,enable electric lights inside the office,O O O O O B-location:room,6
13164,13164,turnLightOn,also enable lights within the lavatory,O O O O O B-location:room,6


In [84]:
df.to_csv('/home/h4zzkr/lab/delOS/data/nlu_data/custom2/train.csv')